In [ ]:
import requests

link = "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en"
f = requests.get(link)
k = (f.text).split("\n")
k = k[:20000]
link = "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de"
f = requests.get(link)
k1 = (f.text).split("\n")
k1 = k1[:20000]

In [ ]:
import pandas as pd
df = pd.DataFrame({'trg':k,'src':k1})
df.head()

,trg,src
0,iron cement is a ready for use paste which is ...,iron cement ist eine gebrauchs ##AT##-##AT## f...
1,iron cement protects the ingot against the hot...,Nach der Aushärtung schützt iron cement die Ko...
2,"a fire restant repair cement for fire places ,...",feuerfester Reparaturkitt für Feuerungsanlagen...
3,Construction and repair of highways and ...,Der Bau und die Reparatur der Autostraßen ...
4,An announcement must be commercial character .,die Mitteilungen sollen den geschäftlichen kom...


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(len(k))
df.to_csv('/content/drive/MyDrive/machine_translation/train.csv',index=False)

20000


In [2]:
!pip install torchtext==0.6.0 --quiet
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset
import numpy as np
import pandas as pd
import spacy, random     
from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary     

     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 1.2MB 3.8MB/s 


In [4]:
## Loading the SpaCy's vocabulary for our desired languages. 
!python -m spacy download en --quiet
!python -m spacy download de --quiet

spacy_german = spacy.load("de")
spacy_english = spacy.load("en")

def tokenize_german(text):
  return [token.text for token in spacy_german.tokenizer(text)]

def tokenize_english(text):
  return [token.text for token in spacy_english.tokenizer(text)]

german = Field(tokenize=tokenize_german, lower=True,
               init_token="<sos>", eos_token="<eos>")

english = Field(tokenize=tokenize_english, lower=True,
               init_token="<sos>", eos_token="<eos>")

fields = [("trg", english),("src", german)]

train_data, valid_data = TabularDataset.splits(
    path='/content/drive/MyDrive/machine_translation',
    train='train.csv', validation='valid.csv',
    format='csv',
    skip_header=True,
    fields=fields)

german.build_vocab(train_data)
english.build_vocab(train_data)

print(f"Unique tokens in source (de) vocabulary: {len(german.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(english.vocab)}")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
Unique tokens in source (de) vocabulary: 48191
Unique tokens in target (en) vocabulary: 29091


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator, valid_iterator = BucketIterator.splits((train_data, valid_data), 
                                                                      batch_size = BATCH_SIZE, 
                                                                      sort_within_batch=True,
                                                                      sort_key=lambda x: len(x.src),
                                                                      device = device)

count=0
for data in train_data:
  if count < 1 :
    print("German - ",*data.src)
    print("English - ",*data.trg)
    print()
  break

for data in train_iterator:
  if count < 1 :
    print("German - ",*data.src)
    print("English - ",*data.trg)
    print()
  break

German -  iron cement ist eine gebrauchs # # at##-##at # # fertige paste , die mit einem spachtel oder den fingern als hohlkehle in die formecken ( winkel ) der stahlguss -kokille aufgetragen wird .
English -  iron cement is a ready for use paste which is laid as a fillet by putty knife or finger in the mould edges ( corners ) of the steel ingot mould .

German -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([   10,  2302,    15,    53,    60,    55,   142,     8,     8,    12,
          176,   111,   306,    13,  2937,    66,   380, 27556,     9, 39258,
            9,  1278,    60,   443,     9,    13,     8,  5376,    71,   372,
            8,  1995], device='cuda:0') tensor([ 1066,    63,  2322, 10331,   222,    63,    42,  3964,  7193,    56,
            7,  3862,    33,   576,   257,  8185,   631,     7, 11577,     7,
        21631,   163,   222,     7, 24717,   119,  3624,     7,    12,   

In [6]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state

input_size_encoder = len(german.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(48191, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [7]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(hidden_size, output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 4556) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = len(english.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(english.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(29091, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=29091, bias=True)
)


In [8]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # Shape - Source : (10, 32) [(Sentence length German + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(english.vocab)
    
    # Shape --> outputs (14, 32, 5766) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    hidden_state, cell_state = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs

In [9]:
learning_rate = 0.001
writer = SummaryWriter(f"runs/loss_plot")
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(48191, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(29091, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=29091, bias=True)
  )
)

In [10]:
def translate_sentence(model, sentence, german, english, device, max_length=50):
    spacy_ger = spacy.load("de")

    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)
    text_to_indices = [german.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [english.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, '/content/checkpoint-NMT')
    torch.save(model.state_dict(),'/content/checkpoint-NMT-SD')

In [13]:
len(train_iterator)

625

In [14]:
epoch_loss = 0.0
num_epochs = 100
best_loss = 999999
best_epoch = -1
sentence1 = "ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster"
ts1  = []

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  translated_sentence1 = translate_sentence(model, sentence1, german, english, device, max_length=50)
  print(f"Translated example sentence 1: \n {translated_sentence1}")
  ts1.append(translated_sentence1)

  model.train(True)
  for batch_idx, batch in enumerate(train_iterator):
    if batch_idx%100==0:
        print(batch_idx)
    input = batch.src.to(device)
    target = batch.trg.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()
    step += 1
    epoch_loss += loss.item()
    writer.add_scalar("Training loss", loss, global_step=step)

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss / len(train_iterator))

score = bleu(valid_data[1:100], model, german, english, device)
print(f"Bleu score {score*100:.2f}")

Epoch - 1 / 100
Translated example sentence 1: 
 ['the', 'guest', 'is', 'the', 'to', 'the', 'the', ',', 'the', ',', 'the', '.', '<eos>']
0
100
200
300
400
500
600
saving

Epoch_Loss - 5.800578594207764

Epoch - 2 / 100
Translated example sentence 1: 
 ['the', 'is', 'is', 'available', 'in', 'the', 'entire', 'of', 'and', 'costs', 'eur', 'per', '.', '.', '<eos>']
0
100
200
300
400
500
600
Epoch_Loss - 5.208927631378174

Epoch - 3 / 100
Translated example sentence 1: 
 ['the', 'is', 'is', 'a', 'in', 'the', ',', 'and', 'the', '.', '<eos>']
0
100
200
300
400
500
600
Epoch_Loss - 5.657285690307617

Epoch - 4 / 100
Translated example sentence 1: 
 ['this', 'is', 'is', 'a', 'the', 'the', 'the', 'the', 'the', 'the', '.', '<eos>']
0
100
200
300
400
500
600
Epoch_Loss - 5.686106204986572

Epoch - 5 / 100
Translated example sentence 1: 
 ['a', 'small', '#', '#', 'at##-##at', '#', '#', 'star', 'hotel', 'is', 'is', 'located', 'in', 'the', 'heart', 'of', 'the', ',', 'the', 'the', 'the', 'the', 'the', 

KeyboardInterrupt: ignored

In [ ]:
model.eval()
test_sentences  = ["Zwei Männer gehen die Straße entlang", "Kinder spielen im Park.", 
                   "Diese Stadt verdient eine bessere Klasse von Verbrechern. Der Spaßvogel"]

actual_sentences  = ["Two men are walking down the street", "Children play in the park", 
                     "This city deserves a better class of criminals. The joker"]
pred_sentences = []

for idx, i in enumerate(test_sentences):
  model.eval()
  translated_sentence = translate_sentence(model, i, german, english, device, max_length=50)
  pred_sentences.append(TreebankWordDetokenizer().detokenize(translated_sentence))
  print("German : {}".format(i))
  print("Actual Sentence in English : {}".format(actual_sentences[idx]))
  print("Predicted Sentence in English : {}".format(pred_sentences[-1]))
  print()

In [16]:
count = 0
for data in train_iterator:
  if count < 1 :
    temp_ger = data.src
    temp_eng = data.trg
    count += 1

In [17]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_ger_idx = (temp_ger).cpu().detach().numpy()

In [18]:
df_eng_idx = pd.DataFrame(data = temp_eng_idx, columns = [str("S_")+str(x) for x in np.arange(1, 33)])
df_eng_idx.index.name = 'Time Steps'
df_eng_idx.index = df_eng_idx.index + 1 
# df_eng_idx.to_csv('/content/idx.csv')
df_eng_idx

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,4,4,69,8,4,4,70,24,34,78,69,125,4662,4,5,47,102,644,4,4,2888,301,2262,1309,26256,4,31,27227,225,2928,1404,4
3,184,116,101,44,116,116,231,281,57,739,101,704,3051,541,804,193,905,35,116,116,2992,4,6,85,9087,96,3149,64,9,4,302,1741
4,76,160,18,31,160,160,44,354,178,5,18,11,8,2435,4,1824,2507,113,160,160,8,441,1107,119,2766,38,5,71,1554,1395,11,9
5,8,20,95,1601,20,20,192,13,619,15,95,1199,10495,3849,444,22,11,19,20,20,11,71,13,16,2288,127,8,471,4631,821,13373,1389
6,737,241,10,98,241,241,422,12,8,37,10,19,9,612,9,4,196,11,241,241,10266,273,56,10326,20605,5,8889,351,53,10,330,25597
7,16,33,61,8198,33,33,1186,9731,548,17,61,993,2321,10543,4,4077,266,10444,33,33,219,6,1107,27,250,4,10,16,2289,91,5,4
8,35,42,28,24461,42,42,8,5,2861,73,28,289,30,5,750,377,9,118,42,42,4317,3,16,1305,1385,177,86,239,53,6,29,5995
9,281,169,4,8,169,169,36,28,6,12314,4,6,24820,270,4076,33,169,715,169,169,794,1,2533,1399,27729,38,99,100,24583,3,15,811


In [20]:
e = list(english.vocab.__dict__.values())
word_2_idx = dict(e[3])
idx_2_word = {}
for k,v in word_2_idx.items():
  idx_2_word[v] = k

In [21]:
df_eng_word = pd.DataFrame(columns = [str("S_")+str(x) for x in np.arange(1, 33)])
df_eng_word = df_eng_idx.replace(idx_2_word)
# df_eng_word.to_csv('/content/Words.csv')
df_eng_word

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>
2,the,the,when,and,the,the,only,this,all,he,when,some,improves,the,",",we,these,start,the,the,increasing,view,innovation,taking,schöne,the,be,supercard,systems,sets,luxembourg,the
3,best,guest,would,will,guest,guest,then,holiday,rooms,said,would,feature,reliability,coast,contains,now,problems,your,guest,guest,diversity,the,.,time,anlage,service,faithful,-,of,the,being,presence
4,offers,reviews,you,be,reviews,reviews,will,home,offer,",",you,a,and,gets,the,focus,affect,day,reviews,reviews,and,top,freedom,out,mit,was,",",new,energy,controller,a,of
5,and,are,like,happy,are,are,guests,is,comfort,&,like,balcony,probability,slightly,level,on,a,with,are,are,a,new,is,for,sehr,great,and,credit,feed,module,landlocked,artists
6,prices,submitted,to,about,submitted,submitted,receive,in,and,quot,to,with,of,less,of,the,small,a,submitted,submitted,mounting,features,also,oneself,freundlichem,",",yield,card,/,to,country,reinforces
7,for,by,stay,inspiring,by,by,keys,facinas,contemporary,;,stay,stunning,detection,rainfall,the,ratings,number,plentiful,by,by,work,.,freedom,or,personal,the,to,for,servers,use,",",the
8,your,our,at,partnerships,our,our,and,",",décor,his,at,views,(,",",error,given,of,breakfast,our,our,load,<eos>,for,someone,und,staff,no,those,/,.,it,bohemian
9,holiday,customers,the,and,customers,customers,can,at,.,diabetic,the,.,pod,while,raised,by,customers,buffet,customers,customers,ask,<pad>,enterprise,else,tollen,was,b,who,peripherals,<eos>,&,spirit
